In [8]:
import pandas as pd
import numpy as np
import datetime as dt
import h5py
import math as ma
from astropy.io import fits

In [2]:
df = pd.read_csv("/Users/eormacstudio/Documents/GitHub/GRANDWin/data/raw/observation_id/observation_id.csv", header=0, engine='python')
df['date'] = df.starttime_utc.apply(lambda x: dt.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.000Z").date())
df['date_time'] = df.starttime_utc.apply(lambda x: dt.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.000Z"))
df['partition'] = pd.factorize(df['date'])[0] + 1
df = df[df['partition'] == 1].reset_index(drop=True)

---

#### Checking the results normal vs faster algorithm

In [ ]:
df_sel_rfi_fast = pd.DataFrame()
transform_tb = np.repeat(np.arange(0,14,1), 4)

for n in range(3):
    with h5py.File('/Users/eormacstudio/Documents/GitHub/GRANDWin/test/results/faster/win_z_scores_data_day_1_grid_%s_integration_8_real.h5' %(n), "r") as f:
        data = f["wins_z_score"][:]
        obs_id = f["obs_id"][:]
        time_blocks = f["time_blocks"][:]

    flat_data = data.reshape(-1, data.shape[-1])
    
    # Create DataFrame
    df_temp = pd.DataFrame(flat_data, columns=["XX", "XY", "YX", "YY"])

    df_temp["obs_id"] = np.tile(obs_id, data.shape[1]*data.shape[2])
    df_temp["time_index"] = np.repeat(np.arange(data.shape[0]), data.shape[1] * data.shape[2])
    df_temp["frequency"] = np.tile(np.arange(data.shape[2]), data.shape[0] * data.shape[1])
    df_temp["antenna"] = np.tile(np.repeat(np.arange(data.shape[1]), data.shape[2]), data.shape[0])
    df_temp["obs_id"] = df_temp['time_index'].map(lambda t: obs_id[t])
    df_temp["timeblock"] = df_temp['time_index'].map(lambda t: int(time_blocks[t]))
    #df_temp['timeblock'] = df_temp.timeblock.apply(lambda x: np.where(transform_tb == x)[0].tolist())
    #df_temp = df_temp.explode('timeblock').reset_index(drop=True)

    df_sel_rfi_fast = pd.concat([df_sel_rfi_fast, df_temp]).reset_index(drop = True)


In [ ]:
df_sel_rfi_slow = pd.DataFrame()
for o in range(3):
    aa = pd.read_csv("/Users/eormacstudio/Documents/GitHub/GRANDWin/test/results/slower/both_side_selected_outliers_3_sigma_day_1_grid_[%s]_iter_100_real_part_8s.csv" %(o), header=None)
    aa.columns = ['xx', 'yy', 'obs_id', 'timeblocks', 'tile', 'freqs', 'z_score', 'polarization']
    aa = aa[~aa['z_score'].isna()].reset_index(drop=True)
    aa['z_score'] = aa['z_score'].astype(float)
    aa['floor_z_score'] = aa['z_score'].apply(lambda x: ma.floor(x))
    df_sel_rfi_slow = pd.concat([df_sel_rfi_slow, aa]).reset_index(drop = True)

In [ ]:
np.unique(df_sel_rfi_fast['obs_id'])

In [ ]:
df_sel_rfi_fast[df_sel_rfi_fast['obs_id'] == 1095449728].head(10)

In [ ]:
df_sel_rfi_slow[(df_sel_rfi_slow['obs_id'] == 1095449728) & (df_sel_rfi_slow['freqs'] == 0) & (df_sel_rfi_slow['tile'] == 0) & (df_sel_rfi_slow['timeblocks'] == 0)]

---

#### Min max comparison

In [ ]:
df_stat_slow = pd.DataFrame()
for o in range(3):
    oo = pd.read_csv("/Users/eormacstudio/Documents/GitHub/GRANDWin/test/results/slower/both_side_outliers_statistics_3_sigma_day_1_grid_[%s]_iter_100_real_part_8s.csv" %(o), header=None)
    oo.columns = ['tile', 'freqs', 'polarization', 'skewness', 'max', 'min', 'mean', 'win_mean', 'std', 'win_std', 'gamma_final', 'number_of_outliers']
    oo['grid'] = o
    df_stat_slow = pd.concat([df_stat_slow, oo]).reset_index(drop = True)

In [ ]:
df_stat_fast = pd.DataFrame()
for o in range(3):
    oo = pd.read_parquet("/Users/eormacstudio/Documents/GitHub/GRANDWin/test/results/faster/outlier_statistics_day_1_grid_%s_integration_8_real.parquet" %(o))
    oo['grid'] = o
    df_stat_fast = pd.concat([df_stat_fast, oo]).reset_index(drop = True)

In [ ]:
df_gamma_fast = pd.DataFrame()
for o in range(3):
    oo = pd.read_parquet("/Users/eormacstudio/Documents/GitHub/GRANDWin/test/results/faster/final_gamma_day_1_grid_%s_integration_8_real.parquet" %(o))
    oo['grid'] = o
    df_gamma_fast = pd.concat([df_gamma_fast, oo]).reset_index(drop = True)

In [ ]:
df_stat_slow[(df_stat_slow['tile'] == 0) & (df_stat_slow['freqs'] == 8)]

In [ ]:
df_stat_fast[(df_stat_fast['antenna'] == 0) & (df_stat_fast['frequency'] == 8) & (df_stat_fast['polarization'].isin([0,3]))]

In [ ]:
df_gamma_fast[(df_gamma_fast['antenna'] == 0) & (df_gamma_fast['frequency'] == 7) & (df_stat_fast['polarization'].isin([0,3]))]

---

In [3]:
df_sel_rfi_fast = pd.DataFrame()
transform_tb = np.repeat(np.arange(0,14,1), 4)

for n in range(1):
    with h5py.File('/Users/eormacstudio/Documents/GitHub/GRANDWin/data/processed/detected_outliers/win_z_scores_data_day_1_grid_%s_integration_8_real.h5' %(n), "r") as f:
        data = f["wins_z_score"][:]
        obs_id = f["obs_id"][:]
        time_blocks = f["time_blocks"][:]

    flat_data = data.reshape(-1, data.shape[-1])
    
    # Create DataFrame
    df_temp = pd.DataFrame(flat_data, columns=["XX", "XY", "YX", "YY"])

    df_temp["obs_id"] = np.tile(obs_id, data.shape[1]*data.shape[2])
    df_temp["time_index"] = np.repeat(np.arange(data.shape[0]), data.shape[1] * data.shape[2])
    df_temp["frequency"] = np.tile(np.arange(data.shape[2]), data.shape[0] * data.shape[1])
    df_temp["antenna"] = np.tile(np.repeat(np.arange(data.shape[1]), data.shape[2]), data.shape[0])
    df_temp["obs_id"] = df_temp['time_index'].map(lambda t: obs_id[t])
    df_temp["timeblock"] = df_temp['time_index'].map(lambda t: int(time_blocks[t]))
    #df_temp['timeblock'] = df_temp.timeblock.apply(lambda x: np.where(transform_tb == x)[0].tolist())
    #df_temp = df_temp.explode('timeblock').reset_index(drop=True)

    df_sel_rfi_fast = pd.concat([df_sel_rfi_fast, df_temp]).reset_index(drop = True)


In [7]:
df_sel_rfi_fast[df_sel_rfi_fast['time_index'] == 8]

,XX,XY,YX,YY,obs_id,time_index,frequency,antenna,timeblock
655360,NaN,NaN,NaN,NaN,1095450704,8,0,0,8
655361,NaN,NaN,NaN,NaN,1095450704,8,1,0,8
655362,NaN,NaN,NaN,NaN,1095450704,8,2,0,8
655363,NaN,NaN,NaN,NaN,1095450704,8,3,0,8
655364,NaN,NaN,NaN,NaN,1095450704,8,4,0,8
...,...,...,...,...,...,...,...,...,...
737275,NaN,NaN,NaN,NaN,1095450704,8,635,127,8
737276,NaN,NaN,NaN,NaN,1095450704,8,636,127,8
737277,NaN,NaN,NaN,NaN,1095450704,8,637,127,8
737278,NaN,NaN,NaN,NaN,1095450704,8,638,127,8


In [10]:
f = fits.open(f"/Users/eormacstudio/Documents/GitHub/GRANDWin/data/raw/calibration_solutions/20250712_observation_data_8s_before_with_cotter_time_corrections_without_flags/uvfits/calibrated_fits/hyperdrive_solutions_1095450704.fits")

In [16]:
f['SOLUTIONS'].data[:, :, :, ::2]

array([[[[            nan,             nan,             nan,
                      nan],
         [            nan,             nan,             nan,
                      nan],
         [            nan,             nan,             nan,
                      nan],
         ...,
         [            nan,             nan,             nan,
                      nan],
         [            nan,             nan,             nan,
                      nan],
         [            nan,             nan,             nan,
                      nan]],

        [[            nan,             nan,             nan,
                      nan],
         [            nan,             nan,             nan,
                      nan],
         [            nan,             nan,             nan,
                      nan],
         ...,
         [            nan,             nan,             nan,
                      nan],
         [            nan,             nan,             nan,
                   

In [17]:
g = fits.open(f"/Users/eormacstudio/Documents/GitHub/GRANDWin/data/raw/calibration_solutions/20240831_multiple_simulation_higher_thermal_noise_sn3_8s/uvfits/calibrated_fits/hyperdrive_solutions_1095450704.fits")

In [18]:
g['SOLUTIONS'].data[:, :, :, ::2]

array([[[[ 1.02169395e+00, -1.52470216e-01,  1.69545726e-01,
           1.07822737e+00],
         [ 1.15155657e+00,  2.09155173e-02,  1.41727100e-01,
           1.03155191e+00],
         [ 9.85908107e-01, -5.29557953e-03, -2.37289984e-01,
           9.32814897e-01],
         ...,
         [ 1.03136666e+00,  2.16737550e-01, -1.33112173e-01,
           1.25863433e+00],
         [ 1.00268181e+00,  7.16321648e-03, -1.21015243e-01,
           1.02236975e+00],
         [ 9.43673702e-01, -2.86236254e-01,  3.47950311e-01,
           9.43368665e-01]],

        [[ 9.55807017e-01, -5.57715815e-02,  1.03718637e-01,
           8.69549818e-01],
         [ 1.04329136e+00,  2.47187568e-03,  3.01009436e-02,
           9.99821534e-01],
         [ 1.00335503e+00,  4.03927416e-02,  6.08742223e-02,
           1.10737542e+00],
         ...,
         [ 1.05445395e+00, -1.80412807e-01, -1.51411214e-01,
           8.80189505e-01],
         [ 1.00568565e+00, -1.90249179e-01, -2.05107653e-01,
           1.109103